In [1]:
#BERT code is from https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

In [ ]:
!pip install bert-tensorflow

In [2]:
import pandas as pd
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [3]:
df_tweets_cleaned = pd.read_csv('..\data\Tweets_cleaned.csv')

In [4]:
df_tweets_cleaned

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,...,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,@VirginAmerica What @dhepburn said.,What said.,What said.,False,...,False,[],What said.,[],What said.,False,what said.,what said.,said,say
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experien...,plus you've added commercials to the experien...,False,...,False,[],plus you've added commercials to the experien...,[],plus you've added commercials to the experien...,False,plus you've added commercials to the experien...,plus you've added commercials to the experien...,plus added commercials experience tacky,plus add commercial experience tacky
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today... Must mean I need to take an...,I didn't today... Must mean I need to take an...,False,...,False,[],I didn't today... Must mean I need to take an...,[],I didn't today... Must mean I need to take an...,False,i didn't today... must mean i need to take an...,i didn't today... must mean i need to take an...,today mean need trip,today mean need trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",False,...,False,[],"it's really aggressive to blast obnoxious ""en...",[],"it's really aggressive to blast obnoxious ""en...",False,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",aggressive blast obnoxious entertainment guest...,aggressive blast obnoxious entertainment guest...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,and it's a really big bad thing about it,False,...,False,[],and it's a really big bad thing about it,[],and it's a really big bad thing about it,False,and it's a really big bad thing about it,and it's a really big bad thing about it,big bad thing,big bad thing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,@AmericanAir thank you we got on a different f...,thank you we got on a different flight to Chi...,thank you we got on a different flight to Chi...,False,...,False,[],thank you we got on a different flight to Chi...,[],thank you we got on a different flight to Chi...,False,thank you we got on a different flight to chi...,thank you we got on a different flight to chi...,thank got different flight chicago,thank get different flight chicago
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,@AmericanAir leaving over 20 minutes Late Flig...,leaving over 20 minutes Late Flight. No warni...,leaving over 20 minutes Late Flight. No warni...,False,...,False,[],leaving over 20 minutes Late Flight. No warni...,[],leaving over 20 minutes Late Flight. No warni...,False,leaving over 20 minutes late flight. no warni...,leaving over 20 minutes late flight. no warni...,leaving minutes late flight warnings communica...,leave minute late flight warning communication...
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,@AmericanAir Please bring American Airlines to...,Please bring American Airlines to #BlackBerry10,Please bring American Airlines to #BlackBerry10,False,...,False,[],Please bring American Airlines to #BlackBerry10,['blackberry10'],Please bring Ame

In [5]:
#df_tweets_bert = df_tweets_cleaned[['airline_sentiment', 'text_cleaned']]
df_tweets_bert = df_tweets_cleaned[['airline_sentiment', 'text_list_no_stop_words']]

In [6]:
df_tweets_bert

,airline_sentiment,text_list_no_stop_words
0,neutral,said
1,positive,plus added commercials experience tacky
2,neutral,today mean need trip
3,negative,aggressive blast obnoxious entertainment guest...
4,negative,big bad thing
...,...,...
14635,positive,thank got different flight chicago
14636,negative,leaving minutes late flight warnings communica...
14637,neutral,bring american airlines
14638,negative,money change flight answer phones suggestions ...


In [7]:
df_tweets_bert['binary_response_variable'] = False

df_tweets_bert.loc[df_tweets_cleaned.airline_sentiment == 'neutral', 'binary_response_variable'] = False
df_tweets_bert.loc[df_tweets_cleaned.airline_sentiment == 'positive', 'binary_response_variable'] = False
df_tweets_bert.loc[df_tweets_cleaned.airline_sentiment == 'negative', 'binary_response_variable'] = True

C:\Users\alexf\Anaconda3\envs\CSML1010_OnlineSession2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\alexf\Anaconda3\envs\CSML1010_OnlineSession2\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
df_tweets_bert.binary_response_variable = df_tweets_bert.binary_response_variable.astype(int)

C:\Users\alexf\Anaconda3\envs\CSML1010_OnlineSession2\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df_tweets_bert = df_tweets_bert[['text_list_no_stop_words', 'binary_response_variable']]
df_tweets_bert

,text_list_no_stop_words,binary_response_variable
0,said,0
1,plus added commercials experience tacky,0
2,today mean need trip,0
3,aggressive blast obnoxious entertainment guest...,1
4,big bad thing,1
...,...,...
14635,thank got different flight chicago,0
14636,leaving minutes late flight warnings communica...,1
14637,bring american airlines,0
14638,money change flight answer phones suggestions ...,1


In [10]:
X = df_tweets_bert.loc[:, df_tweets_bert.columns != 'binary_response_variable']
Y = df_tweets_bert.loc[:, df_tweets_bert.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

In [11]:
X_train

,text_list_no_stop_words
4771,arrangements reimburse rental
12454,apologizing rude sales reps failure offer trit...
3840,year old flying tokyo vacation bad knees happens
14176,baggage lost flight cancelled flightled accomm...
1219,passengers rerouted match intended arrival tim...
...,...
7233,yeah aware original message
3855,free upgrade problems reservation mkwlkr
6339,hey guys honest customers unlike
11606,jfk baggage office open help book cancelled fl...


In [12]:
XY_train = pd.concat([X_train, Y_train], axis=1)
XY_test = pd.concat([X_test, Y_test], axis=1)

In [13]:
XY_train = XY_train.dropna()
XY_test = XY_test.dropna()

In [14]:
print(XY_train.isnull().values.any())
print(XY_test.isnull().values.any())

False
False


In [15]:
DATA_COLUMN = 'text_list_no_stop_words'
LABEL_COLUMN = 'binary_response_variable'

# label_list is the list of labels. In this situation they are 0, 1
label_list = [0, 1]

In [16]:
XY_train

,text_list_no_stop_words,binary_response_variable
4771,arrangements reimburse rental,0
12454,apologizing rude sales reps failure offer trit...,1
3840,year old flying tokyo vacation bad knees happens,1
14176,baggage lost flight cancelled flightled accomm...,1
1219,passengers rerouted match intended arrival tim...,1
...,...,...
7233,yeah aware original message,1
3855,free upgrade problems reservation mkwlkr,1
6339,hey guys honest customers unlike,0
11606,jfk baggage office open help book cancelled fl...,1


## Data Preprocessing

In [17]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = XY_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = XY_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [18]:
train_InputExamples

4771     <bert.run_classifier.InputExample object at 0x...
12454    <bert.run_classifier.InputExample object at 0x...
3840     <bert.run_classifier.InputExample object at 0x...
14176    <bert.run_classifier.InputExample object at 0x...
1219     <bert.run_classifier.InputExample object at 0x...
                               ...                        
7233     <bert.run_classifier.InputExample object at 0x...
3855     <bert.run_classifier.InputExample object at 0x...
6339     <bert.run_classifier.InputExample object at 0x...
11606    <bert.run_classifier.InputExample object at 0x...
3899     <bert.run_classifier.InputExample object at 0x...
Length: 10190, dtype: object

In [19]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [20]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 10190


INFO:tensorflow:Writing example 0 of 10190


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [21]:
train_InputExamples

4771     <bert.run_classifier.InputExample object at 0x...
12454    <bert.run_classifier.InputExample object at 0x...
3840     <bert.run_classifier.InputExample object at 0x...
14176    <bert.run_classifier.InputExample object at 0x...
1219     <bert.run_classifier.InputExample object at 0x...
                               ...                        
7233     <bert.run_classifier.InputExample object at 0x...
3855     <bert.run_classifier.InputExample object at 0x...
6339     <bert.run_classifier.InputExample object at 0x...
11606    <bert.run_classifier.InputExample object at 0x...
3899     <bert.run_classifier.InputExample object at 0x...
Length: 10190, dtype: object

## Model

In [22]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [23]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
#         f2_score = tf.contrib.metrics.f2_score(
#             label_ids,
#             predicted_labels
#         )
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
#             "f2_score": f2_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [24]:
BATCH_SIZE = 1
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0

WARMUP_PROPORTION = 0.1

SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [25]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [26]:
OUTPUT_DIR = 'bert'

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [27]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'bert', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000272098F33C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'bert', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000272098F33C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [29]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


Training took time  0:00:00.048015


In [30]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [43]:
#print(f'Beginning predictions!')
#current_time = datetime.now()
estimator.evaluate(input_fn=test_input_fn, steps=None)
#print("Training predictions took this amount of time ", datetime.now() - current_time)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\Users\alexf\Anaconda3\envs\CSML1010_OnlineSession2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 30570: auc = 0.81412077, eval_accuracy = 0.83272314, f1_score = 0.86944085, false_negatives = 311.0, false_positives = 420.0, global_step = 30570, loss = 1.1803308, precision = 0.8528381, recall = 0.8867031, true_negatives = 1205.0, true_positives = 2434.0


INFO:tensorflow:Saving dict for global step 30570: auc = 0.81412077, eval_accuracy = 0.83272314, f1_score = 0.86944085, false_negatives = 311.0, false_positives = 420.0, global_step = 30570, loss = 1.1803308, precision = 0.8528381, recall = 0.8867031, true_negatives = 1205.0, true_positives = 2434.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30570: bert\model.ckpt-30570


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30570: bert\model.ckpt-30570


{'auc': 0.81412077,
 'eval_accuracy': 0.83272314,
 'f1_score': 0.86944085,
 'false_negatives': 311.0,
 'false_positives': 420.0,
 'loss': 1.1803308,
 'precision': 0.8528381,
 'recall': 0.8867031,
 'true_negatives': 1205.0,
 'true_positives': 2434.0,
 'global_step': 30570}

In [37]:

def getPrediction(in_sentences):
  labels = [0, 1]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [41]:
#pred_sentences = [
#  "My flight to New York was delayedn AGAIN"#,
#  #"So excited to be on the flight to San Francisco!"
#]

#predictions = getPrediction(pred_sentences)
#predictions

INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


IndexError: tuple index out of range

# References

Code for this project is either directly from (with some modification), or inspired by, but not limited to the following sources:

* Predicting Movie Review Sentiment with BERT on TF Hub:
https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
* Introducing BERT with Tensorflow: 
https://www.kaggle.com/sergeykalutsky/introducing-bert-with-tensorflow